# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv ../../01_materials/labs/.env

In [2]:
import dask.dataframe as dd

/var/folders/_r/1p823h7d1jj507k0d71gjk6w0000gn/T/ipykernel_5106/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

PRICE_DATA = os.getenv('PRICE_DATA')

# Use glob to find all parquet files recursively
parquet_files = glob(os.path.join(PRICE_DATA, '**', '*.parquet'), recursive=True)
# Make sure we only have files
parquet_files = [f for f in parquet_files if os.path.isfile(f)]


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Reading the parquet files into a dask object
dData = dd.read_parquet(parquet_files).set_index('ticker')
print(dData)

# Option 1: Add features using chain of apply(), lambda, and assign()
dd_feat = (
    dData.groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.assign(
                closeLag = x['Close'].shift(1),
                adjCloseLag = x['Adj Close'].shift(1),
            )
        )
        .assign(
            # Calculate the daily high-low range
            hi_lo_range = lambda x: x['High'] - x['Low']
        )
        .assign(
            # Calculate returns based on Adjusted Close
            returns = lambda x: x['Adj Close'] / x['adjCloseLag'] - 1
        )
)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/var/folders/_r/1p823h7d1jj507k0d71gjk6w0000gn/T/ipykernel_5106/259607445.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dData.groupby('ticker', group_keys=False)


Dask DataFrame Structure:
                             Date     Open     High      Low    Close Adj Close Volume  sector subsector   year
npartitions=11207                                                                                              
                   datetime64[ns]  float64  float64  float64  float64   float64  int64  string    string  int32
                              ...      ...      ...      ...      ...       ...    ...     ...       ...    ...
...                           ...      ...      ...      ...      ...       ...    ...     ...       ...    ...
                              ...      ...      ...      ...      ...       ...    ...     ...       ...    ...
                              ...      ...      ...      ...      ...       ...    ...     ...       ...    ...
Dask Name: read-parquet, 1 graph layer


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [5]:
import pandas as pd

pData = dd_feat.compute() # This compute applies the calculations written in the lambda and return a Pandas Dataframe but it takes tiiiime (1.5 minutes in my local)


In [6]:
pData['rolling_avg_return'] = pData.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
pData

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,closeLag,adjCloseLag,hi_lo_range,returns,rolling_avg_return
ticker,,,,,,,,,,,,,,,
HUM,2002-01-02,11.800000,11.830000,11.270000,11.500000,10.296815,572500,Health Care,Managed Health Care,2002,NaN,NaN,0.559999,NaN,NaN
HUM,2002-01-03,11.500000,11.690000,11.380000,11.500000,10.296815,693600,Health Care,Managed Health Care,2002,11.500000,10.296815,0.309999,0.000000,NaN
HUM,2002-01-04,11.550000,11.600000,11.310000,11.550000,10.341583,651900,Health Care,Managed Health Care,2002,11.500000,10.296815,0.290000,0.004348,NaN
HUM,2002-01-07,11.540000,11.660000,11.400000,11.480000,10.278908,576400,Health Care,Managed Health Care,2002,11.550000,10.341583,0.260000,-0.006061,NaN
HUM,2002-01-08,11.400000,11.560000,11.280000,11.460000,10.260998,406500,Health Care,Managed Health Care,2002,11.480000,10.278908,0.280001,-0.001742,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2020-12-24,22.459999,22.459999,22.219999,22.400000,20.193138,758000,Information Technology,Communications Equipment,2020,22.340000,20.139053,0.240000,0.002686,0.001973
JNPR,2020-12-28,22.500000,22.930000,22.430000,22.559999,20.337374,2005800,Information Technology,Communications Equipment,2020,22.400000,20.193138,0.500000,0.007143,0.002915
JNPR,2020-12-29,22.620001,22.709999,22.090000,22.190001,20.003832,2130000,Information Technology,Communications Equipment,2020,22.559999,20.337374,0.619999,-0.016400,0.000545


In [7]:
# This code is in charge to print a message in case of an error in the calculation of the 10 days rolling avg
nan_counts = pd.DataFrame(pData.groupby('ticker')['rolling_avg_return'].apply(lambda x: x.isna().sum()))
filtered_nan_counts = nan_counts[nan_counts['rolling_avg_return'] != 10]

if filtered_nan_counts.shape[0] > 0:
    print(f"there are {filtered_nan_counts.shape[0]} tickers with more than 10 NaNs in the rolling 10 avg, please check") 


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1. It wasn't necessary to convert the Dask dataframe in order to calculate the moving average since we can do it also in Dask.
1. It all depends on the context, in this particular scenario the number of records that we're trying to calculate is not in the order of millions, then Pandas perform pretty decent, when dealing with several millions of rows the performance of Pandas will be slower than a potential parallel execution using Dask.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.